# 8.9 연습



리스트8.26 Google 드라이브상의 파일로의 경로를 지정해 파일을 읽어 들인다

In [10]:
from google.colab import drive

drive.mount('/content/drive/')

# Google 드라이브상의 텍스트 데이터로의 경로를 지정하세요.
nov_path = '/content/drive/MyDrive/' + 'Colab Notebooks/AK_Colab/chapter_08/novels/은하철도의 밤.txt'

# 파일을 읽어 들인다
with open(nov_path, 'r') as f:
    nov_text = f.read()
    print(nov_text[:1000])  # 첫 1000문자만 표시

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
"그럼, 여러분은 이렇게 강이라고 하거나 우유가 흐른 흔적이라고 말하고 있는 이 희미하고 하얀 것이 실제로는 무엇인지 알고 있습니까?" 선생님은 칠판에 걸어 놓은 커다란 검은 별자리 지도의 위에서 아래쪽으로 희뿌연 띠 모양을 한 은하를 가리키며 모두에게 질문을 던졌습니다.
캄파넬라가 손을 들었습니다. 그리고 나서 네다섯 명이 손을 더 들었습니다. 조바니도 손을 들려고 하다가 황급히 그대로 멈추었습니다. 분명 그것이 모두 별이라고 언젠가 잡지에서 읽었지만, 요즘은 조바니는 교실에서도 졸고, 책을 읽을 틈도 읽을 책도 없기 때문에 왠지 아는 게 아무것도 없는 듯한 기분이 들었습니다.
그런데 선생님은 벌써 그것을 눈치챘습니다.
"조바니, 너는 알고 있겠지?"
조바니는 기세 좋게 일어났지만 막상 일어나보니 분명하게 대답할 수가 없었습니다. 자네리가 앞자리에서 뒤돌아보고 조바니를 보고 키득키득 웃었습니다. 조바니는 더 당황해서 얼굴이 새빨개졌습니다. 선생님이 또 말했습니다.
"큰 망원경으로 은하를 잘 살펴보면 은하는 대체 무엇일까요?"
역시 별이라고 조바니는 생각했지만 이번에도 바로 대답할 수 없었습니다.
선생님은 잠시 난처한 기색이었지만 눈을 캄파넬라 쪽으로 향하고,
"그럼 캄파넬라"라고 지명했습니다. 그러자 그렇게 씩씩하게 손을 든 캄파넬라가 역시 쭈뼛거리며 일어선 채 결국 대답할 수 없었습니다.
선생님은 의외라는 듯 잠시 물끄러미 캄파넬라를 보았지만, 서둘러 "그럼. 좋아요."라고 말하면서 스스로 별자리 그림을 가리켰습니다.
"이 희미한 은하를 성능이 좋은 망원경으로 보면 무수히 많은 작은 별로 보이는 겁니다. 조바니 그렇지요?"
조바니는 새빨간 얼굴로 끄덕였습니다. 그렇지만 어느새 조바니의 눈에는 눈물이 가득 고였습니다. 
'그래 

8-9-2 정규 표현에 의한 전처리

In [11]:
import re  # 정규 표현에 필요한 라이브러리

text = re.sub("《[^》]+》", "", nov_text) # 루피 삭제
text = re.sub("［[^］]+］", "", text) # 읽기 주의의 삭제
text = re.sub("[｜ 　]", "", text) # | 와 스페이스의 삭제
print("문자 수", len(text))  # len()로 문자열의 문자 수도 취득 가능

문자 수 1721


리스트8.28 RNN의 각 설정

In [12]:
n_rnn = 10  # 시계열의 수
batch_size = 128
epochs = 60
n_mid = 128  # 중간층의 뉴런 수

리스트8.29 문자의 벡터화

In [14]:
import numpy as np

# 인덱스와 문자로 사전을 작성
chars = sorted(list(set(text)))  # set으로 문자의 중복을 없이 각 문자를 리스트에 저장한다
print("문자 수(중복 없음)", len(chars))
char_indices = {}  # 문자가 키로 인덱스가 값
for i, char in enumerate(chars):
    char_indices[char] = i
indices_char = {}  # 인덱스가 키로 문자가 값
for i, char in enumerate(chars):
    indices_char[i] = char

# 시계열로 늘어선 문자와 앞으로 예측해야 할 문자를 꺼낸다  
time_chars = []  # 시계열로 늘어선 문자
next_chars = []  # 예측해야 할 문자
for i in range(0, len(text) - n_rnn):
    time_chars.append(text[i: i + n_rnn])
    next_chars.append(text[i + n_rnn])

# 입력과 정답을 one-hot 표현으로 나타냅니다
x = np.zeros((len(time_chars), n_rnn, len(chars)), dtype=np.bool_)  # 입력
t = np.zeros((len(time_chars), len(chars)), dtype=np.bool_)  # 정답
for i, t_cs in enumerate(time_chars):
    t[i, char_indices[next_chars[i]]] = 1  # 정답을 one-hot 표현으로 나타낸다
    for j, char in enumerate(t_cs):
        x[i, j, char_indices[char]] = 1  # 입력을 one-hot 표현으로 나타낸다
        
print("x의 형태", x.shape)
print("t의 형태", t.shape)

문자 수(중복 없음) 301
x의 형태 (1711, 10, 301)
t의 형태 (1711, 301)


리스트8.30 모델의 구축

In [18]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, SimpleRNN, LSTM, GRU

model = Sequential()
# ----- 다음에 코드를 추기한다 -----



리스트8.31 문자 생성용의 함수

In [19]:
from tensorflow.python.keras.callbacks import LambdaCallback
 
def on_epoch_end(epoch, logs):
    print("에포크: ", epoch)

    beta = 5  # 확률 분포를 조정하는 상수
    prev_text = text[0:n_rnn]  # 입력에 사용하는 문자
    created_text = prev_text  # 생성되는 텍스트
    
    print("시드: ", created_text)

    for i in range(400):
        # 입력을 one-hot 표현으로
        x_pred = np.zeros((1, n_rnn, len(chars)))
        for j, char in enumerate(prev_text):
            x_pred[0, j, char_indices[char]] = 1
        
        # 예측을 실시, 다음 문자를 얻는다
        y = model.predict(x_pred)
        p_power = y[0] ** beta  # 확률 분포의 조정
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power))        
        next_char = indices_char[next_index]

        created_text += next_char
        prev_text = prev_text[1:] + next_char

    print(created_text)
    print()

# 에포크 종료 후에 실행되는 함수를 설정 
epock_end_callback= LambdaCallback(on_epoch_end=on_epoch_end)

리스트8.32 학습

In [22]:
model = model
history = model.fit(x, t,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[epock_end_callback])

Epoch 1/60
14/14 [==============================] - 3s 40ms/step - loss: 5.6924
에포크:  0
시드:  "그럼,여러분은이렇
"그럼,여러분은이렇렌렌판친인릴니체얗까난록스,띠갈유며작뼛닥기할시냐또므향시섯'푸해강든옆세저엇할푸살카트우부태네멈님자엾도교음실손제앞밖욱멀생습생득벌당뼛경든흐욱안은구교르틈웃결닥그말원않얆슷흐아쌍약트섯의신파섯짜갔태났얆림태생론더바생크작함공며교추곧얆전펼녁많갔
것언키슷크나떠찰않서.연속덮려양앞역능려르름전으즈치눈띠건두밤얇물당선혀스대빛부럼작단잘갖아치가떠신국트까분희판,했읽한와면사굴하른께우에면런정수래선세친님있둘면났빼경단켰데습론덮뜩사학카판이푸은께빨손외얀까칠일딜웃데태안새그마다야록거은의대불니른건멈거파빨왠유까진우습듯의굴찰욱였쭈자부했많되견유크손노나시너역뒤를떠태려관스르공강한상진색하네자면잡릿침시재제실왠로자써름'러본겠구칠을카좋.릿야명번짝가손못개쪽녁잘엇틈각네건크성음래인란씩났럼국제형어란들씩명모득오엇도런딜알저엾공불여즉판펼속엾깊떠푸수더놓서향므옆어섯론버여"구불박듯대전트쌍희늘태

Epoch 2/60
14/14 [==============================] - 1s 38ms/step - loss: 5.6071
에포크:  1
시드:  "그럼,여러분은이렇
"그럼,여러분은이렇이틈이간국우앞멈기급도버살면그침서잘라부그분있놓캄쭈네러여하갖라하을를은좋에지름푸미듯어고잊그래볼한시으.않밤속르조빼버했면이은친모졌태하각겁에스성라거지닥일보지"니책바이더그그속파미므니캄하하인렇가거.각면지기생선대전유그이저님시공것었졌버잡문문졌입면들시님알습로있저무닥개문떠보은지다은니그즉즈로어커이무닫많만그즘리도트이진그데엇씩이의해하그.빛별만본자즈신그운았일다그속이득희노한한비님지요실도읽푸이고면를고책유를얼미생와은으얗는는를도'께진럼았.그제하로책닫르라이번국나칠으여그공없살무엇오하즘른깊"게보조없이에를는모에있그이뼛이그한그이반이.읽그기라하어찰겁시그이니느대모견"에이그조만이씩그쪽로생흔없이은를어생,라을는이마는생에이판희
는읽미를이여이네를는습물않상알이는로능뜩이이신.알생라님득.그.입책라번살보얗아은파므렇운보

8.10 정답 예


리스트8.33 정답 예

In [21]:
model = Sequential()
# ----- 다음에 코드를 추기한다 -----
model.add(GRU(n_mid, input_shape=(n_rnn, len(chars))))
model.add(Dense(len(chars), activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer="adam")
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 128)               165120    
_________________________________________________________________
dense (Dense)                (None, 301)               38829     
Total params: 203,949
Trainable params: 203,949
Non-trainable params: 0
_________________________________________________________________
None
